# Bank Note Authentication 

## Abstract:
    
Data were extracted from images that were taken for the evaluation of an authentication procedure for banknotes.

## Data Set Information:
    
Data were extracted from images that were taken from genuine and forged banknote-like specimens. For digitization, an 
industrial camera usually used for print inspection was used. The final images have 400x 400 pixels. Due to the object lens and
distance to the investigated object gray-scale pictures with a resolution of about 660 dpi were gained. Wavelet Transform tool
were used to extract features from images.

Source: https://archive.ics.uci.edu/ml/datasets/banknote+authentication

## Attribute Information:
    
variance - variance of Wavelet Transformed image (continuous)
skewness - skewness of Wavelet Transformed image (continuous)
curtosis - curtosis of Wavelet Transformed image (continuous)
entropy - entropy of image (continuous)
class - class (integer)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#let us start by importing the relevant libraries
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#reading the banknote dataset in a dataframe. 
columns = ["var","skewness","curtosis","entropy","class"]
df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/00267/\
data_banknote_authentication.txt",index_col=False, names = columns)

In [ ]:
df

In [ ]:
# we separate the target variable (class) and save it in the y variable. Also the X contains the independant variables.
X = df.iloc[:,0:4].values
y = df.iloc[:,4].values


In [ ]:
#splitting the data in test and train sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 10)

In [ ]:
# scaling the data using the standard scaler
from sklearn.preprocessing import StandardScaler
X_train_sd = StandardScaler().fit_transform(X_train)
X_test_sd = StandardScaler().fit_transform(X_test)

In [ ]:
# generating the covariance matrix and the eigen values for the PCA analysis
cov_matrix = np.cov(X_train_sd.T) # the relevanat covariance matrix
print('Covariance Matrix \n%s', cov_matrix)

In [ ]:
#generating the eigen values and the eigen vectors, backened working of PCA
e_vals, e_vecs = np.linalg.eig(cov_matrix)
print('Eigenvectors \n%s' %e_vecs)
print('\nEigenvalues \n%s' %e_vals)

In [ ]:
# ACTUAL PCA ALGOL
from sklearn.decomposition import PCA
pca = PCA(n_components=4)
pca.fit(X_train_sd)

In [ ]:
print(pca.components_)

In [ ]:
print(pca.explained_variance_)

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
X_train_sd

In [ ]:
X_train_sd_pca = pca.transform(X_train_sd)
X_train_sd_pca

In [ ]:
(-0.24410388*X_train_sd[:,0])+(-0.63914113*X_train_sd[:,1])+(0.61378454*X_train_sd[:,2])+(0.3939295*X_train_sd[:,3])

In [ ]:
# the "cumulative variance explained" analysis 
tot = sum(e_vals)
var_exp = [( i /tot ) * 100 for i in sorted(e_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print("Cumulative Variance Explained", cum_var_exp)

In [ ]:
# Plotting the variance expalained by the principal components and the cumulative variance explained.
plt.figure(figsize=(10 , 5))
plt.bar(range(1, e_vals.size + 1), var_exp, alpha = 0.5, align = 'center', label = 'Individual explained variance')
plt.step(range(1, e_vals.size + 1), cum_var_exp, where='mid', label = 'Cumulative explained variance')
plt.ylabel('Explained Variance Ratio')
plt.xlabel('Principal Components')
plt.legend(loc = 'best')
plt.tight_layout()
plt.show()

In [ ]:
eigen_pairs = [(np.abs(e_vals[i]), e_vecs[:,i]) for i in range(len(e_vals))]
eigen_pairs.sort(reverse=True)
eigen_pairs[:5]

In [ ]:
w = np.hstack((eigen_pairs[0][1].reshape(4,1), 
                      eigen_pairs[1][1].reshape(4,1)))
w

In [ ]:
# generating dimensionally reduced datasets
w = np.hstack((eigen_pairs[0][1].reshape(4,1), 
                      eigen_pairs[1][1].reshape(4,1)))
print('Matrix W:\n', w)
X_sd_pca = X_train_sd.dot(w)
X_test_sd_pca = X_test_sd.dot(w)

In [ ]:
X_train_sd.shape, w.shape, X_sd_pca.shape, X_test_sd_pca.shape

### We will use Logistic regression, RandomForest and AdaBoost

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_sd, y_train)
print ('Before PCA score', model.score(X_test_sd, y_test))

model.fit(X_sd_pca, y_train)
print ('After PCA score', model.score(X_test_sd_pca, y_test))


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train_sd, y_train)
print ('Before PCA score', clf.score(X_test_sd, y_test))

clf.fit(X_sd_pca, y_train)
print ('After PCA score', clf.score(X_test_sd_pca, y_test))


In [ ]:
### In the given dataset we trained models with the orginal and dimensionally reduced datsets. The effects of PCA can be clearly appreciated on a fairly large datsaset. The learners are encouraged to try the above with various large datsets out there.

In [ ]:
## Reference Link 
* https://www.analyticsvidhya.com/blog/2020/12/an-end-to-end-comprehensive-guide-for-pca/